Prepard by : Bryan Lim Yuqiang  <br> 

Notice: please pip install oandapyV20 before using

<a href="https://www.oanda.com/register/?div=3&cc=SG#/sign-up/demo" /><button style = "background-color: green; color: white;"> Sign up for your demo account</button> </a>
<a href="https://developer.oanda.com/rest-live-v20/introduction/" /><button style = "background-color: green; color: white;"> OandaV20 API Website </button> </a>
<a href="https://buildmedia.readthedocs.org/media/pdf/oanda-api-v20/latest/oanda-api-v20.pdf" /><button style = "background-color: green; color: white;"> Detailed API Documentation </button> </a>
<a href="https://developer.oanda.com/rest-live-v20/troubleshooting-errors/" /><button style = "background-color: red; color: white;"> Troubleshooting & Errors </button> </a>

In [2]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades
import pandas as pd

### API SETUP - Generate your own api key and store to txt file in a directory

In [3]:
#initiating API connection and definining trade parameters 
token_path =  "C:\\Users\\Gigabyte\\Desktop\\quant-trading-python-bot\\oanda.txt"
client = oandapyV20.API(access_token = open(token_path,'r').read(), environment='practice')

### Retrieving Historical Data using instruments endpoint

In [4]:
""" instruments endpoints is involved in retrieving historical data """
params  = {'count': 150, 'granularity': 'M5'}  #<- refer to instrument definitions
candles = instruments.InstrumentsCandles(instrument='USD_JPY', params = params)
client.request(candles)
print(candles.response)
ohlc_dict = candles.response['candles']
ohlc = pd.DataFrame(ohlc_dict)  # convert json data into dataframe
ohlc_df = ohlc.mid.dropna().apply(pd.Series)  # splits o,h,l,c into a new df
ohlc_df['volume'] = ohlc['volume'] 
ohlc_df.index = ohlc['time']
ohlc_df = ohlc_df.apply(pd.to_numeric) #original in string, needs to convert to numeric


{'instrument': 'USD_JPY', 'granularity': 'M5', 'candles': [{'complete': True, 'volume': 281, 'time': '2020-05-08T08:30:00.000000000Z', 'mid': {'o': '106.320', 'h': '106.346', 'l': '106.314', 'c': '106.332'}}, {'complete': True, 'volume': 144, 'time': '2020-05-08T08:35:00.000000000Z', 'mid': {'o': '106.334', 'h': '106.352', 'l': '106.324', 'c': '106.332'}}, {'complete': True, 'volume': 179, 'time': '2020-05-08T08:40:00.000000000Z', 'mid': {'o': '106.333', 'h': '106.333', 'l': '106.308', 'c': '106.308'}}, {'complete': True, 'volume': 230, 'time': '2020-05-08T08:45:00.000000000Z', 'mid': {'o': '106.308', 'h': '106.318', 'l': '106.304', 'c': '106.315'}}, {'complete': True, 'volume': 182, 'time': '2020-05-08T08:50:00.000000000Z', 'mid': {'o': '106.314', 'h': '106.334', 'l': '106.310', 'c': '106.332'}}, {'complete': True, 'volume': 184, 'time': '2020-05-08T08:55:00.000000000Z', 'mid': {'o': '106.333', 'h': '106.360', 'l': '106.332', 'c': '106.357'}}, {'complete': True, 'volume': 168, 'time':

### Streaming Live Data 

In [9]:
""" ----------------------- Streaming data -------------------------  """


params = {'instruments': 'USD_JPY'}
account_id = '101-003-14701906-001' #log in to oanda terminal to see your own account id
r = pricing.PricingInfo(accountID = account_id, params = params)
i = 0
n = 2 # number of response you want , adjust this to your own preference
while i <= n: 
    rv = client.request(r)
    print('Time=', rv['time'])
    print('bid=', rv['prices'][0]['closeoutBid'])
    print('ask=', rv['prices'][0]['closeoutAsk'])
    print('************************************')
    i+=1

# feel free to check out the response by only printing rv

Time= 2020-05-09T16:22:40.514530939Z
bid= 106.588
ask= 106.692
************************************
Time= 2020-05-09T16:22:41.514500080Z
bid= 106.588
ask= 106.692
************************************
Time= 2020-05-09T16:22:41.514500080Z
bid= 106.588
ask= 106.692
************************************


### Check out Account Details

In [10]:
""" -------------------------Trading Account Details------------------------------""" 
r = accounts.AccountDetails(accountID = account_id)
client.request(r)
print(r.response) 
#check out balance, hedging, currency

{'account': {'guaranteedStopLossOrderMode': 'ALLOWED', 'hedgingEnabled': False, 'id': '101-003-14701906-001', 'createdTime': '2020-05-09T12:54:41.604833253Z', 'currency': 'SGD', 'createdByUserID': 14701906, 'alias': 'Primary', 'marginRate': '0.02', 'lastTransactionID': '3', 'balance': '100000.0000', 'openTradeCount': 0, 'openPositionCount': 0, 'pendingOrderCount': 0, 'pl': '0.0000', 'resettablePL': '0.0000', 'resettablePLTime': '0', 'financing': '0.0000', 'commission': '0.0000', 'dividendAdjustment': '0', 'guaranteedExecutionFees': '0.0000', 'orders': [], 'positions': [], 'trades': [], 'unrealizedPL': '0.0000', 'NAV': '100000.0000', 'marginUsed': '0.0000', 'marginAvailable': '100000.0000', 'positionValue': '0.0000', 'marginCloseoutUnrealizedPL': '0.0000', 'marginCloseoutNAV': '100000.0000', 'marginCloseoutMarginUsed': '0.0000', 'marginCloseoutPositionValue': '0.0000', 'marginCloseoutPercent': '0.00000', 'withdrawalLimit': '100000.0000', 'marginCallMarginUsed': '0.0000', 'marginCallPerc

### Check out account summary (Quite similar to Account Details)

In [11]:
r = accounts.AccountSummary(accountID = account_id)
client.request(r)
print(r.response)

{'account': {'guaranteedStopLossOrderMode': 'ALLOWED', 'hedgingEnabled': False, 'id': '101-003-14701906-001', 'createdTime': '2020-05-09T12:54:41.604833253Z', 'currency': 'SGD', 'createdByUserID': 14701906, 'alias': 'Primary', 'marginRate': '0.02', 'lastTransactionID': '3', 'balance': '100000.0000', 'openTradeCount': 0, 'openPositionCount': 0, 'pendingOrderCount': 0, 'pl': '0.0000', 'resettablePL': '0.0000', 'resettablePLTime': '0', 'financing': '0.0000', 'commission': '0.0000', 'dividendAdjustment': '0', 'guaranteedExecutionFees': '0.0000', 'unrealizedPL': '0.0000', 'NAV': '100000.0000', 'marginUsed': '0.0000', 'marginAvailable': '100000.0000', 'positionValue': '0.0000', 'marginCloseoutUnrealizedPL': '0.0000', 'marginCloseoutNAV': '100000.0000', 'marginCloseoutMarginUsed': '0.0000', 'marginCloseoutPositionValue': '0.0000', 'marginCloseoutPercent': '0.00000', 'withdrawalLimit': '100000.0000', 'marginCallMarginUsed': '0.0000', 'marginCallPercent': '0.00000'}, 'lastTransactionID': '3'}


### Creating Orders   (refer to pg 45 of API booklet)

In [18]:
data = {
    'order': {
        'price':'1.15',
        'stopLossOnFill':{    # 
            'timeInForce': 'GTC', # Good Till Cancelled
            'price': '1.2'
        },
        'timeInForce': 'FOK', # google timeInForce, FOK = Fill or Kill
        'instrument': 'USD_JPY',
        'units': '100', # if -100 = sell order
        'type': 'MARKET',
        'positionFill':'DEFAULT'
    }
}

In [19]:
r = orders.OrderCreate(accountID=account_id, data = data)
client.request(r)

{'orderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'USD_JPY',
  'units': '100',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'stopLossOnFill': {'price': '1.200', 'timeInForce': 'GTC'},
  'reason': 'CLIENT_ORDER',
  'id': '9',
  'accountID': '101-003-14701906-001',
  'userID': 14701906,
  'batchID': '9',
  'requestID': '60708125655353688',
  'time': '2020-05-09T16:52:50.229411429Z'},
 'orderCancelTransaction': {'type': 'ORDER_CANCEL',
  'orderID': '9',
  'reason': 'MARKET_HALTED',
  'id': '10',
  'accountID': '101-003-14701906-001',
  'userID': 14701906,
  'batchID': '9',
  'requestID': '60708125655353688',
  'time': '2020-05-09T16:52:50.229411429Z'},
 'relatedTransactionIDs': ['9', '10'],
 'lastTransactionID': '10'}

### Customising the Orders to our flexibility

In [27]:
def ATR(DF,n):
    """Function to calculate True Range and Average True Range"""
    # remember n is number of period points
    df= DF.copy()
    df['H-L'] = abs(df['h'] - df['l'])
    df['H-PC'] = abs(df['h'] - df['c'].shift(1))
    df['L-PC'] = abs(df['l'] - df['c'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1, skipna= False)
    df['ATR'] = df['TR'].rolling(n).mean()
    #df['ATR'] = df['TR'].ewm(span = n, adjust = False, min_period = n).mean()
    df2 = df.drop(['H-L', 'H-PC','L-PC'], axis = 1)
    return round(df2['ATR'][-1], 2)

In [30]:
def  market_order(account_id, instrument, units, sl):
    """ units can be +ve or -ve, stop loss(in pips) added/subtracted to price"""
    params = {'instruments' : instrument}
    r = pricing.PricingInfo(accountID = account_id, params = params)
    rv = client.request(r)
     
    #sl here refers to stop loss in pip and is determined by ATR as defined above
    if units > 0:
        price = float(rv['prices'][0]['closeoutAsk'])
        st_ls = price - sl
    else: 
        price = float(rv['prices'][0]['closeoutBid'])
        st_ls = price + sl 
        
        
    data = {
        'order':{
            'price': "",
            'stopLossOnFill':{
                'timeInForce':'GTC',
                'price': str(st_ls)
            },
            'timeInForce':'FOK',
            'instrument': str(instrument),
            'units': str(units),
            'type': 'MARKET',
            'positionFill' : 'DEFAULT'
        }
    }
    
    return data

In [32]:
r = orders.OrderCreate(accountID = account_id, data = market_order(account_id, 'USD_JPY', -100, 3*ATR(ohlc_df,120)))
client.request(r)

{'orderCreateTransaction': {'type': 'MARKET_ORDER',
  'instrument': 'USD_JPY',
  'units': '-100',
  'timeInForce': 'FOK',
  'positionFill': 'DEFAULT',
  'stopLossOnFill': {'price': '106.708', 'timeInForce': 'GTC'},
  'reason': 'CLIENT_ORDER',
  'id': '12',
  'accountID': '101-003-14701906-001',
  'userID': 14701906,
  'batchID': '12',
  'requestID': '24679337667747447',
  'time': '2020-05-09T17:28:43.750520251Z'},
 'orderCancelTransaction': {'type': 'ORDER_CANCEL',
  'orderID': '12',
  'reason': 'MARKET_HALTED',
  'id': '13',
  'accountID': '101-003-14701906-001',
  'userID': 14701906,
  'batchID': '12',
  'requestID': '24679337667747447',
  'time': '2020-05-09T17:28:43.750520251Z'},
 'relatedTransactionIDs': ['12', '13'],
 'lastTransactionID': '13'}

In [33]:
### Check Trades
r = trades.OpenTrades(accountID = account_id)
client.request(r)

{'trades': [], 'lastTransactionID': '13'}